In [12]:
using LinearAlgebra
using CairoMakie
using SparseArrays
using Symbolics

In [13]:
# Test points to test all functions
x0 = [-1.71, 1.59, 1.82, -0.763, -0.763]
λ0 = ones(3)

3-element Vector{Float64}:
 1.0
 1.0
 1.0

In [14]:
# Function to be optimized
function f(x)
    return exp(x[1]*x[2]*x[3]*x[4]*x[5]) - ((x[1]^3 + x[2]^3 + 1)^2) / 2
end

# Gradient of the function to be optimized
function grad_f(x::Vector{T}) where T <: Number
    ex1 = exp(x[1]*x[2]*x[3]*x[4]*x[5])
    ex2 = (x[1]^3 + x[2]^3 + 1)
    grad_f = zeros(5)
    grad_f[1] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[2]*x[3]*x[4]*x[5] - (3//2)*(2 + 2(x[1]^3) + 2(x[2]^3))*(x[1]^2)
    grad_f[2] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[1]*x[3]*x[4]*x[5] - (3//2)*(2 + 2(x[1]^3) + 2(x[2]^3))*(x[2]^2)
    grad_f[3] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[1]*x[2]*x[4]*x[5]
    grad_f[4] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[1]*x[2]*x[3]*x[5]
    grad_f[5] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[1]*x[2]*x[3]*x[4]
    return grad_f
end

function hess_f(x::Vector{T}) where T <: Number
    hess_f = zeros(5, 5)
    hess_f[1,1] = 2((-3//1) - (3//1)*(x[1]^3) - (3//1)*(x[2]^3))*x[1] + (x[2]^2)*(x[3]^2)*(x[4]^2)*(x[5]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5]) - (9//1)*(x[1]^4)
    hess_f[1,2] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[3]*x[4]*x[5] + (x[3]^2)*(x[4]^2)*(x[5]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[1]*x[2] - (9//1)*(x[1]^2)*(x[2]^2)
    hess_f[1,3] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[2]*x[4]*x[5] + (x[2]^2)*(x[4]^2)*(x[5]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[1]*x[3]
    hess_f[1,4] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[2]*x[3]*x[5] + (x[2]^2)*(x[3]^2)*(x[5]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[1]*x[4]
    hess_f[1,5] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[2]*x[3]*x[4] + (x[2]^2)*(x[3]^2)*(x[4]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[1]*x[5]
    hess_f[2,1] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[3]*x[4]*x[5] + (x[3]^2)*(x[4]^2)*(x[5]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[1]*x[2] - (9//1)*(x[1]^2)*(x[2]^2)
    hess_f[2,2] = 2((-3//1) - (3//1)*(x[1]^3) - (3//1)*(x[2]^3))*x[2] + (x[1]^2)*(x[3]^2)*(x[4]^2)*(x[5]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5]) - (9//1)*(x[2]^4)
    hess_f[2,3] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[1]*x[4]*x[5] + (x[1]^2)*(x[4]^2)*(x[5]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[2]*x[3]
    hess_f[2,4] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[1]*x[3]*x[5] + (x[1]^2)*(x[3]^2)*(x[5]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[2]*x[4]
    hess_f[2,5] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[1]*x[3]*x[4] + (x[1]^2)*(x[3]^2)*(x[4]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[2]*x[5]
    hess_f[3,1] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[2]*x[4]*x[5] + (x[2]^2)*(x[4]^2)*(x[5]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[1]*x[3]
    hess_f[3,2] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[1]*x[4]*x[5] + (x[1]^2)*(x[4]^2)*(x[5]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[2]*x[3]
    hess_f[3,3] = (x[1]^2)*(x[2]^2)*(x[4]^2)*(x[5]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5])
    hess_f[3,4] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[1]*x[2]*x[5] + (x[1]^2)*(x[2]^2)*(x[5]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[3]*x[4]
    hess_f[3,5] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[1]*x[2]*x[4] + (x[1]^2)*(x[2]^2)*(x[4]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[3]*x[5]
    hess_f[4,1] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[2]*x[3]*x[5] + (x[2]^2)*(x[3]^2)*(x[5]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[1]*x[4]
    hess_f[4,2] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[1]*x[3]*x[5] + (x[1]^2)*(x[3]^2)*(x[5]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[2]*x[4]
    hess_f[4,3] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[1]*x[2]*x[5] + (x[1]^2)*(x[2]^2)*(x[5]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[3]*x[4]
    hess_f[4,4] = (x[1]^2)*(x[2]^2)*(x[3]^2)*(x[5]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5])
    hess_f[4,5] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[1]*x[2]*x[3] + (x[1]^2)*(x[2]^2)*(x[3]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[4]*x[5]
    hess_f[5,1] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[2]*x[3]*x[4] + (x[2]^2)*(x[3]^2)*(x[4]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[1]*x[5]
    hess_f[5,2] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[1]*x[3]*x[4] + (x[1]^2)*(x[3]^2)*(x[4]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[2]*x[5]
    hess_f[5,3] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[1]*x[2]*x[4] + (x[1]^2)*(x[2]^2)*(x[4]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[3]*x[5]
    hess_f[5,4] = exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[1]*x[2]*x[3] + (x[1]^2)*(x[2]^2)*(x[3]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5])*x[4]*x[5]
    hess_f[5,5] = (x[1]^2)*(x[2]^2)*(x[3]^2)*(x[4]^2)*exp(x[1]*x[2]*x[3]*x[4]*x[5])
    return hess_f
end

f(x0)
grad_f(x0)
hess_f(x0)

5×5 Matrix{Float64}:
 -76.5943     -66.6435    -0.0976501   0.232927   0.232927
 -66.6435     -57.5232     0.10502    -0.250506  -0.250506
  -0.0976501    0.10502    0.140529   -0.218849  -0.218849
   0.232927    -0.250506  -0.218849    0.799578   0.522025
   0.232927    -0.250506  -0.218849    0.522025   0.799578

In [15]:
#------------------------------------------------------
# Constraints and its Jacobians and Hessians
#------------------------------------------------------
# Function to be optimized
function g(x::Vector{T}) where T
    v = zeros(3)
    v[1] = x[1]^2 + x[2]^2 + x[3]^2 + x[4]^2 + x[5]^2 - 10
    v[2] = x[2]*x[3] - 5*x[4]*x[5]
    v[2] = x[1]^3 + x[2]^3 + 1
    return v
end

function jac_g(x::Vector{T}) where T
    jac_g = zeros(3, 5)
    jac_g[1, 1] = 2*x[1] 
    jac_g[1, 2] = 2*x[2]
    jac_g[1, 3] = 2*x[3]
    jac_g[1, 4] = 2*x[4]
    jac_g[1, 5] = 2*x[5]
    jac_g[2, 1] = 0
    jac_g[2, 2] = x[3]
    jac_g[2, 3] = x[2]
    jac_g[2, 4] = -5*x[5]
    jac_g[2, 5] = -5*x[4]
    jac_g[3, 1] = 3*x[1]^2
    jac_g[3, 2] = 3*x[2]^2
    jac_g[3, 3] = 0
    jac_g[3, 4] = 0
    jac_g[3, 5] = 0
    return jac_g
end

function jac_g_sparse(x::Vector{T}) where T
    return [2*x[1], 2*x[2], 2*x[3], 2*x[4], 2*x[5], x[3], x[2], -5*x[5], -5*x[4],
            3*x[1]^2, 3*x[2]^2]
end
rindg = [1 1 1 1 1 2 2 2 2 3 3]
cindg = [1 2 3 4 5 2 3 4 5 1 2]

function hess_g1(x::Vector{T}) where T
    return Matrix{Float64}(2.0I, 5, 5)
end

function hess_g2(x::Vector{T}) where T
    return  [0  0  0   0   0;
             0  0  1   0   0;
             0  1  0   0   0;
             0  0  0   0  -5;
             0  0  0  -5   0]
end

function hess_g3(x::Vector{T}) where T
    hess_g3 = zeros(5,5)
    hess_g3[1, 1] = 6x[1]
    hess_g3[2, 2] = 6x[2]
    return hess_g3
end

hess_g3 (generic function with 1 method)

In [16]:
function hess_l(x::Vector{T}, λ::Vector{T}) where T
    return hess_f(x) - λ[1]*hess_g1(x) - λ[2]*hess_g2(x) - λ[3]*hess_g3(x)
end

hess_l(x0, λ0)

5×5 Matrix{Float64}:
 -68.3343     -66.6435    -0.0976501   0.232927   0.232927
 -66.6435     -69.0632    -0.89498    -0.250506  -0.250506
  -0.0976501   -0.89498   -1.85947    -0.218849  -0.218849
   0.232927    -0.250506  -0.218849   -1.20042    5.52202
   0.232927    -0.250506  -0.218849    5.52202   -1.20042

In [17]:
function kkt_mat(x::Vector{T}, λ::Vector{T}) where T
    return [hess_l(x, λ) -jac_g(x)';
            jac_g(x)      I(3)]
end

function kkt_vec(x::Vector{T}) where T
    return [-grad_f(x) ; -g(x)]
end
A = kkt_mat(x0, λ0)
kkt_vec_jac(x) = [-hess_f(x); -jac_g(x)]
kkt_vec_jac(x0)

8×5 Matrix{Float64}:
 76.5943     66.6435     0.0976501  -0.232927  -0.232927
 66.6435     57.5232    -0.10502     0.250506   0.250506
  0.0976501  -0.10502   -0.140529    0.218849   0.218849
 -0.232927    0.250506   0.218849   -0.799578  -0.522025
 -0.232927    0.250506   0.218849   -0.522025  -0.799578
  3.42       -3.18      -3.64        1.526      1.526
 -0.0        -1.82      -1.59       -3.815     -3.815
 -8.7723     -7.5843    -0.0        -0.0       -0.0

In [18]:
# Implementation follows Boyd
function back_track(f_m, f_m_grad, x, p; β = 0.1, α = 0.3)
    t = 1
    while (f_m(x + t*p) > f_m(x) + α*t*f_m_grad(x)'*p)
        t = β*t
    end
    return t
end

# write the function to be used for line search
f_m(x) = sum(kkt_vec(x) .* kkt_vec(x)) / 2
f_m_grad(x) = kkt_vec_jac(x)'*kkt_vec(x)
f_m(x0)


0.08548261157111248

In [19]:
iter_new = 0
n = length(x0) + length(λ0)

max_iter = 500
tol = 1e-2
alpha0 = 1
iter_val = 1

for i=1:max_iter
    iter_val = i
    global λ0, x0
    A = kkt_mat(x0, λ0) 
    b = kkt_vec(x0)

    delta_x0 = A \ b

    alpha0 =  back_track(f_m, f_m_grad, x0, delta_x0[1:length(x0)])
    
    x0 = x0 + alpha0*delta_x0[1:length(x0)]
    λ0 = alpha0*delta_x0[length(x0)+1:end]

    if norm(grad_f(x0))^2 < tol
        break
    end
end


In [20]:
x0


5-element Vector{Float64}:
 -1.7094410837898981
  1.586859067640408
  1.82333488158231
 -0.7629612341569375
 -0.7629612341569375

In [21]:
iter_val

500

In [22]:
λ0
alpha0

1.000000000000001e-16